In [ ]:
import pandas as pd
import seaborn as sns
import h5py
import Classification_Utils as CU
import DataStream_Vis_Utils as utils
import pickle
from Analysis_Utils import preprocessing_df as preprocessing
with h5py.File('RM16_920_S3_exp_pos_data_filt.h5', 'r') as hf:
    blist = list(hf['labels'])
    arms = (hf['timeseries']).value
    probs=(hf['probabilities']).value
    experimental = (hf['experimental_data']).value     



def import_robot_data(df_path):
    with open(df_path,'rb') as f:
        df=pickle.load(f)
    #df = pd.read_hdf(df_path)
    df = preprocessing(df)
    return df
robot_data_path = 'experimental_data.pickle'
hdf = import_robot_data(robot_data_path)
hdf = hdf.reset_index(drop=True)
dd_1 = utils.get_single_trial(hdf,'20','S3','RM16')
dd_1 = dd_1.reset_index(drop=True)
d = CU.pkl_to_df('tkdf16_new.pkl')
#d14=pd.read_hdf('kinematic_data_save_df_RM14.h5')


In [83]:
dd_1.head()

,rat,S,Date,dim,time,SF,t_m,m_start,m_stop,lick,...,RW,r_start,r_stop,r,t2,t1,exp_response,x_pot,y_pot,z_pot
0,RM16,S3,20,/home/pns/rat_reaching/scripts/9pt_pidiv3_cone...,"[19.63243318646755, 19.658433209355735, 19.730...","[1, 2, 3, 4, 6, 7, 8, 10, 17, 18, 19, 20, 21, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39.224433194096946, 105.84643341153469, 124.9...","[43.230433236058616, 111.41943336575832, 128.6...","[0.0, 108.42176666666667, 108.4302, 108.7308, ...",...,0 0 1 0 2 0 3 0 4 ...,"[2753, 12174, 14881, 16186, 17211, 18954, 2314...","[3317, 12957, 15401, 16787, 17765, 19536, 2369...",791.5 683.2 594.5 646.5 531.1 622.1 663.4 534....,521.9 660.5 412.7 561.9 352.7 719.5 522.9 562....,543.4 769.3 559.1 565.3 545.5 616.6 440.4 549....,0 s 1 s 2 s 3 s 4 ...,"[7281, 7273, 7241, 7225, 7265, 7273, 7273, 720...","[8177, 8177, 8185, 8041, 8177, 8177, 8177, 818...","[6969, 6969, 6969, 6881, 6993, 6985, 6961, 673..."


In [84]:
starts = dd_1['r_start'][0]

In [85]:
zx = d['Handle']['X']
palm_r = d['Palm 1']
palm_l = d['Palm 2']

In [86]:
for f in palm_r.index:
    print(f)

('RM16', '0190926', 'S2', 0)
('RM16', '0190926', 'S1', 0)
('RM16', '0190926', 'S4', 0)
('RM16', '0190926', 'S3', 0)
('RM16', '0190928', 'S2', 0)
('RM16', '0190928', 'S1', 0)
('RM16', '0190920', 'S1', 0)
('RM16', '0190920', 'S2', 0)
('RM16', '0190920', 'S4', 0)
('RM16', '0190920', 'S3', 0)
('RM16', '0190927', 'S1', 0)
('RM16', '0190927', 'S2', 0)
('RM16', '0190927', 'S3', 0)
('RM16', '0190927', 'S4', 0)
('RM16', '0190918', 'S1', 0)
('RM16', '0190918', 'S2', 0)
('RM16', '0190918', 'S4', 0)
('RM16', '0190918', 'S3', 0)
('RM16', '0190925', 'S2', 0)
('RM16', '0190925', 'S1', 0)
('RM16', '0190925', 'S3', 0)
('RM16', '0190925', 'S4', 0)
('RM16', '0190919', 'S2', 0)
('RM16', '0190919', 'S1', 0)
('RM16', '0190919', 'S4', 0)
('RM16', '0190919', 'S3', 0)
('RM16', '0190917', 'S1', 0)
('RM16', '0190917', 'S2', 0)
('RM16', '0190917', 'S4', 0)
('RM16', '0190917', 'S3', 0)


In [87]:
import numpy as np
import pdb
def get_reach_indices(DLC_handle,robot_handle,starts,rob=False):
    """
    Function to obtain reach indices from BLOCKS of positional and experimental data.
    """
    
    reach_indices_=np.ones((len(starts)))
    rob_reach_indices = np.zeros((len(starts)))
    #exp_index=DLC_handle.index
    for i,trials in enumerate(starts):
        handle_pos = DLC_handle[trials:trials+500]
        rew_indice = handle_pos_filter(handle_pos)
        if rew_indice == 0:
            reach_indices_[i]= 2
        else:
            reach_indices_[i]=rew_indice
    if rob:
        for ix,rtrials in enumerate(starts):
            robot_pos = robot_handle[starts:starts+500]
            r_ind = np.where((robot_pos) > .15)[0]
            rob_reach_indices[ix] = r_ind
        r_diff = reach_indices_ - rob_reach_indices
    else:
        r_diff = 0
    return reach_indices_




# zero out first 50
def handle_pos_filter(handle_coords,pos_thresh = 0.0):
    ht_ind = np.where((handle_coords[25:-1]) > pos_thresh)
    #pdb.set_trace()
    if ht_ind[0].any():
        first_rew = int(ht_ind[0][0])
        #if first_rew == 0:
            #pdb.set_trace()
        #pdb.set_trace()
        #first_rew=first_rew[0]
    else:
        first_rew = 2 
    return first_rew



def make_first_rew_ind(d,hdf):
    block=[]
    start_16 = []
    licking_list = []
    key=[]
    times = []
    ii=0
    for ifs,z in enumerate(d['Handle']['X'].iteritems()):
#for ii,z in enumerate(d['Handle']):
    # get index, get associated data from exp array
    #pdb.set_trace()
        rat,date,session,dim = z[0]
        z=z[1]
        dd_1 = utils.get_single_trial(hdf,date[-2:],session,rat)
        dd_1 = dd_1.reset_index(drop=True)
        starting_times = dd_1['r_start'][0]
        licking_times = dd_1['lick'].to_numpy()[0]
        time_ = dd_1['time'].to_numpy()[0]
        exp_time = np.asarray(time_, dtype=float) - time_[0]
        lmi = np.searchsorted(exp_time, licking_times)
        lick_mask_array = np.zeros(exp_time.shape[0] + 1, dtype=int)
        lick_mask_array[lmi] = 1
        np.delete(lick_mask_array, [-1])
    #pdb.set_trace()
        rob_x=0
        block.append(get_reach_indices(z,rob_x,starting_times,rob=False))
        start_16.append(starting_times)
        licking_list.append(lick_mask_array)
        times.append(exp_time)
        key.append(z[0])
        ii +=1
    return block,start_16,licking_list,key,times


def autocorr(x):
    result = np.correlate(x, x, mode='full')
    N = len(result)
    half = result[N//2:]
    lengths = range(N, N//2, -1)
    half /= lengths
    half /= half[0]
    return half


def get_speed_palm_trial(p_a,times):
    #pdb.set_trace()
    try:
        p_a = np.diff(p_a,axis=0)
        vp = np.sqrt(((p_a[:,0]/times)**2)+((p_a[:,1])/times)**2+((p_a[:,2])/times)**2)
    except:
        vp = 0
        pdb.set_trace()
    #vp = np.abs((np.mean(,axis=0)) / times 
    all_zeros = not vp.any()
    if all_zeros:
        print('Trial Has 0 Vel')
    return vp , all_zeros


def get_vels_palm_trial(p_arrays_,times):
    #times = times *100 # units of ms
    vs = np.diff(p_arrays_) / times
    all_zeros = not vs.any()
    if all_zeros:
        print('Trial Has 0 Speed for bodypart ' )
    return vs, all_zeros


block16,start16,lick16,key16,times16 = make_first_rew_ind(d,hdf)

C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: in the future negative indices will not be ignored by `numpy.delete`.


In [262]:
def prob_mask_segmentation(palm_probs,feature_array,threshold=0.1,feat_dim=True,palms=True):
    # get indices of probability values under threshold
    palm_thresholds = [6,18] # indices of bodyparts to get threshold vals from
    for prob_array in palm_probs:
        #pdb.set_trace()
        low_pvalue_indices_c1 = np.where(abs(prob_array[0][:]) < threshold)[0]
        low_pvalue_indices_c2 = np.where(abs(prob_array[1][:]) < threshold)[0]
        low_pvalue_indices_c3 = np.where(abs(prob_array[2][:]) < threshold)[0]
        #pdb.set_trace()
        if feat_dim:
            if palms:
                try:
                    f=0
                    for item in np.nditer(low_pvalue_indices_c3): # iter over indices that have low p-values
                        #pdb.set_trace()
                        if f > 2 & item < feature_array[0].shape[0]-5:
                            #print(f)
                            feature_array[:][item] = np.mean(feature_array[:][item-1:item+1],axis=1) # mini interpolation
                            f+=1
                        else:
                            f+=1
                            print(f)
                except:
                    print(f)
                    #pdb.set_trace()
            else:
                for dx in range(0,3):
                    feature_array[dx,:] [low_pvalue_indices_c2] = np.mean(feature_array[item-1:item+1])
                    # camera 3 is underneath and is an excellent way to check if the arm is out
                #feature_array[dx,:] [low_pvalue_indices_c1] = 0
                #feature_array[dx,:] [low_pvalue_indices_c2] = 0
    return feature_array


In [263]:
def get_block_rew_reaches(nb,df,starting_vector,licking_vector,time_vector,keys,length=70):
    # create holder lists
    lha = []
    rha = []
    lhv = []
    rhv = []
    lhs = []
    rhs = []
    lhar = []
    rhar = []
    for ixx,blocks in enumerate(starting_vector): # get blocks
        # Create block data sources
        exp_times = time_vector[ixx]
        licks = licking_vector[ixx]
        indices = nb[ixx]
        key = keys[ixx]
        palm_r = df['Palm 1'].to_numpy()[ixx][0:3]
        palm_l = df['Palm 1'].to_numpy()[ixx][3:6]
        handle = df['Handle'].to_numpy()[ixx][0:3]
        palm_r_probs = df['Palm 2'].to_numpy()[ixx][3:6]
        palm_l_probs = df['Palm 2'].to_numpy()[ixx][0:3]
        # Smooth out low-prob points to decrease the jumping
        palm_r = prob_mask_segmentation([palm_r_probs],palm_r)
        non_reaching_indices = np.where(indices == 2)
        # Create arrays for storing position values
        sl = len(blocks)-(non_reaching_indices[0].shape[0])
       # pdb.set_trace()
        rbp = np.zeros((sl,length,3))
        lbp = np.zeros((sl,length,3))
        hbp = np.zeros((sl,length,3))
        lpv = np.zeros((sl,length-1,3))
        rpv = np.zeros((sl,length-1,3))
        lps = np.zeros((sl,length-1))
        rps = np.zeros((sl,length-1))
        ra = np.zeros((sl,length-1))
        la = np.zeros((sl,length-1))
        ia=0
        for ixs, starts in enumerate(blocks):
            ps = indices[ixs]
            if ps == 2:
                continue
            else:
                pss=int(starts+ps)
                tt = exp_times[pss-length:pss][0:-1]
                for cx in range(0,3):
                    lp = palm_r[cx]
                    rp = palm_l[cx]
                    #pdb.set_trace()
                    rbp[ia,:,cx] = rp[pss-length:pss]
                    lbp[ia,:,cx] = lp[pss-length:pss]
                    rpv[ia,:,cx] = get_vels_palm_trial(rbp[ia,:,cx],tt)[0]
                    lpv[ia,:,cx] = get_vels_palm_trial(lbp[ia,:,cx],tt)[0]
                #pdb.set_trace()
                rps[ia,:] = get_speed_palm_trial(rbp[ia,:,:],tt)[0]
                lps[ia,:] = get_speed_palm_trial(lbp[ia,:,:],tt)[0]
                la[ia,:] = autocorr(lps[ia,:])
                ra[ia,:] = autocorr(rps[ia,:])
                ia+=1
        lha.append(lbp)
        rha.append(rbp)
        lhv.append(lpv)
        rhv.append(rpv)
        lhs.append(rps)
        rhs.append(lps)
        lhar.append(la)
        rhar.append(ra)
    return np.asarray(lha), np.asarray(rha), np.asarray(lhv), np.asarray(rhv), np.asarray(lhs), np.asarray(rhs),np.asarray(lhar), np.asarray(rhar)

pr_90,pl_90,vr90,vl90,sr90,sl90,ra90,la90 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=90)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [142]:
sl90.shape

(30,)

In [284]:
## 
pr_60,pl_60,vr60,vl60,sr60,sl60,ra60,la60 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=60)
pr_70,pl_70,vr70,vl70,sr70,sl70,ra70,la70 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=70)
pr_80,pl_80,vr80,vl80,sr80,sl80,ra80,la80 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=80)
pr_100,pl_100,vr100,vl100,sr100,sl100,ra100,la100 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=100)
pr_150,pl_150,vr150,vl150,sr150,sl150,ra150,la150 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=150)
pr_250,pl_250,vr250,vl250,sr250,sl250,ra250,la250 = get_block_rew_reaches(block16,d,start16,lick16,times16,key16,length=250)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [ ]:
# Look at trace speeds over window for: reaches, non-reaches
# Save the best reach for comparison to empty
length_list = 

In [285]:
# Have 5 X 3 X 3 X T series of reaching
# key must be the length (for establishing good reach windows)

timekey = ['250','100','90','80','70','60']
varkey = ['Right Palm Position','Left Palm Position','Right Palm Velocity','Left Palm Velocity',
          'Right Palm Speed','Left Palm Speed', 'Right Palm AutoCorrelation', 'Left Palm AutoCorrelation','Index']

comp_d = {timekey[0]:[pr_250,pl_250,vr250,vl250,sr250,sl250,ra250,la250,np.full((100),100)],
    timekey[1]:[pr_100,pl_100,vr100,vl100,sr100,sl100,ra100,la100,np.full((100),100)], 
    timekey[2]:[pr_90,pl_90,vr90,vl90,sr90,sl90,ra90,la90,np.full((90),90)], timekey[3]:[pr_80,pl_80,vr80,vl80,sr80,sl80,ra80,la80,np.full((80),80)],
          timekey[4]:[pr_70,pl_70,vr70,vl70,sr70,sl70,ra70,la70,np.full((70),70)], timekey[5]:[pr_60,pl_60,vr60,vl60,sr60,sl60,ra60,la60,np.full((60),60)]      
         }


In [286]:
reaching_df = pd.DataFrame.from_dict(comp_d,orient='index',columns=varkey)
#reaching_df.head()
f=0# f indexes variables (speed is fr =4,5)
for i,comps in comp_d.items():
    c=len(comps) # this accesses 100
    speed_array = comps[4:5]
    #=plt.figure()
    for speeds in speed_array:
        svd = []
        plt.figure()
        plt.title('right hand speeds')
        for ix in range(0,speeds.shape[0]):
            #pdb.set_trace()
            plot_arr = np.mean(speeds[ix],axis=0)
            svd.append(plot_arr)
            plt.plot(plot_arr,label=i)
        plt.figure()
        svd=np.asarray(svd)
        svd = np.nan_to_num(svd,0)
        plt.plot(np.mean(svd,axis=0))
        #pdb.set_trace()
        plt.show()
        plt.legend()
        plt.show()  
            #pdb.set_trace()

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [287]:
fr=0
for i,comps in comp_d.items():
    colors = ['r','b','g','y','m','k']
    c=len(comps) # this accesses 100
    speed_array = comps[6:7]
    for speeds in speed_array:
        block_arr=[]
        plt.figure()
        for ix in range(0,speeds.shape[0]):
                #pdb.set_trace()
                plot_arr = (np.mean(speeds[ix],axis=0))
                # Across a given trial, find each blocks time-averaged signal
                block_arr.append(plot_arr)
                plt.plot(plot_arr,c=colors[fr])
        block_arr=np.asarray(block_arr)
            #pdb.set_trace()
        bp=np.mean(np.nan_to_num(block_arr,0),axis=0)# Rep Across all blocks
    plt.title('Normalized Absolute Value of AutoCorrelation Of Palm Speed Over a Time-Varying Reaching Window')
    plt.legend([i])
    plt.show()
    fr+=1
    
    
plt.figure()
plt.plot(bp,c=colors)
plt.title('autocorrelation of speeds across time windows')
plt.legend(['Time window of 100 Frames','Time window of 90 Frames','Time window of 80 Frames','Time window of 70 Frames','Time window of 60 Frames'])
plt.show()

Traceback (most recent call last):
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\colors.py", line 174, in to_rgba
    rgba = _colors_full_map.cache[c, alpha]
TypeError: unhashable type: 'list'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\backends\backend_qt5.py", line 519, in _draw_idle
    self.draw()
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\backends\backend_agg.py", line 402, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\artist.py", line 50, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\figure.py", line 1649, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-

In [105]:
reaching_df.to_csv('Reach_trials.csv')

In [126]:
reaching_df['Right Palm Speed'].to_numpy()[0].shape

(30,)

In [111]:
reaching_df['Right Palm Speed'].index


Index(['100', '90', '80', '70', '60'], dtype='object')

NameError: name 'handle' is not defined

In [128]:
# check if there is over 4000 frames between r_start and r_stop. If there is, take that frame index for obtaining null
space_after_trials = []
for a,trials in enumerate(starts):
    if a < len(starts)-1:
        if trials + 2000 > starts[a+1]:
            continue
        else:
            space_after_trials.append(a)
# add 2000 to these indices, then slice from here to another 1000 (from 2k-3k frames, we know there is nothing for 4k)

In [129]:
space_after_trials

[0, 1, 5, 11, 14, 16, 27, 28, 29, 32, 33, 34]

In [130]:
# Get block
dd = d[d.index.get_level_values('rat') == 'RM16']
ddd = dd[dd.index.get_level_values('date') == '0190920']
dddd = ddd[ddd.index.get_level_values('session') == 'S3']
# Reduce d to handle, palms, nose 
handle=dddd['Handle'].to_numpy()
rpalm=dddd['Palm 1'].to_numpy()
lpalm=dddd['Palm 2'].to_numpy()
nose = dddd['Nose'].to_numpy()
handle.shape # 30,6 , 30 blocks, 6 dims per block. Access correct block for this session

(1, 6)

In [ ]:
handle[0,1].shape # each is shape nFrames

In [131]:
# Plot 3-D Values 
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig=plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.title('3-D Reconstructed Positions for trial vs explore behavior : Right Palm')
for ax,trialz in enumerate(starts):
    #plt.scatter(handle[0,1][trialz:trialz+400],handle[0,2][trialz:trialz+400],handle[0,3][trialz:trialz+400],
    #            c='b',label='Handle')
    plt.scatter(rpalm[0,1][trialz:trialz+400],rpalm[0,2][trialz:trialz+400],rpalm[0,3][trialz:trialz+400],
                c='b',label='Right Palm')
    #plt.scatter(lpalm[0,0][trialz:trialz+400],lpalm[0,1][trialz:trialz+400],lpalm[0,2][trialz:trialz+400],
    #            c='b',label='Left Palm')
    if ax in space_after_trials:
    #    plt.scatter(handle[0,1][trialz+2000:trialz+2400],handle[0,2][trialz+2000:trialz+2400],handle[0,3][trialz+2000:trialz+2400],
    #            c='r',label='Handle')
        plt.scatter(rpalm[0,1][trialz+2000:trialz+2400],rpalm[0,2][trialz+2000:trialz+2400],rpalm[0,3][trialz+2000:trialz+2400],
                c='r',label='Right Palm')
        #plt.scatter(rpalm[0,3][trialz+2000:trialz+2400],rpalm[0,4][trialz+2000:trialz+2400],rpalm[0,5][trialz+2000:trialz+2400],
        #        c='r',label='Left Palm')
#plt.legend()
# Enforce axis boundaries
#plt.zlim(-0.2, 0.2)
#plt.xlim(-0.2, 0.2)
#plt.ylim(-0.2, 0.2)
plt.xlabel('X (M)')
plt.ylabel('Y (M)')
#plt.zlabel('Z (M)')

plt.show()

C:\Users\bassp\anaconda3\envs\DLC-GPU\lib\site-packages\matplotlib\collections.py:874: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
